# Support Vector Machine
I have finally reached a point in the project where models can be trained using the data pipeline I've created. To start, I am using a support vector machine algorithm. It is known for doing exceptionally well in binary classification and when the feature to sample ratio of the data is relatively high.

I dropped many (possibly useful) columns when processing the data because not all recording use the same channels, this leads to major discrepancies. I think if performance is horrendously abysmal bringing those columns back in elegantly will be a priority. 

It took a while to get to this point, so I want to shout out some of the things I learned/did to get here: spectral entropy, PSD, low pass filtering, time-series data segmentation, band power, tons of data extraction/analysis/preprocessing, time/frequency domain feature extraction, data labeling

### Goals 
- refresh myself on the basic ML train, test, analyze workflow
- gain insight on the performance of an SVM using the data, do some hyperparameter tuning
### Notes


In [6]:
import importlib
from sklearn.svm import SVC

import extraction
from extraction.LT import LTPipeline
importlib.reload(extraction.LT) # bc I keep updating LTPipeine, this ensures the ipynb cache updates
from extraction.tools import get_all_edf_files_for_patient

In [7]:
# pipeline
files = get_all_edf_files_for_patient('chb01')
files.extend(get_all_edf_files_for_patient('chb02'))
files.extend(get_all_edf_files_for_patient('chb03'))
files.extend(['chb15_06.edf'])   # manually add one file that will be validation data

# print(files)
pipeline = LTPipeline(files, verbose=True)
X_train, X_test, y_train, y_test = pipeline.train_test_split(validation_patient_id='chb15')


2025-07-02 20:34:30,205 - INFO - 117 total file(s) in pipeline!


TypeError: LTPipeline.run.<locals>.signal_handler() missing 1 required positional argument: 'frame'

In [ ]:
model = SVC(
    C=1.0,
    tol=0.01,
    kernel="rbf",
    class_weight="balanced",
    random_state=27
)
'''
C - Regularization parameter. The strength of the regularization is inversely proportional to C.
tol - Tolerance for stopping criterion.
class_weight - The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies
in the input data. i.e. higher frequency classes have lower weights (for calculating loss), and vice versa
'''


In [ ]:
y_pred_train_pca = rf.predict(X_train_pca)
acc = accuracy_score(y_train_new, y_pred_train_pca, normalize=True)
print(f'Accuracy for the train data is {acc*100:.2f}%')

